In [35]:
from arff_parser import *
import numpy as np
import sys
import math

In [36]:
class question:
    def __init__(self, feature, threshold, nominal, arffdata):
        #feature is the column number in the data set, nominal=1: feature is nominal
        self.feature = arffdata.attributes[feature].name
        self.threshold = threshold
        self.nominal = nominal
        
    def __repr__(self):
        if self.nominal == 1:
            return "%s == %s ?" % (self.feature, self.threshold)
        if self.nominal == 0:
            return "%s > %s ?" % (self.feature, self.threshold)

In [37]:
a = arff_data("credit_train.arff")
b = a.all_attributes
b[0].attribute_list

['b', 'a']

In [38]:
# for continuous features(nominal=0), instance refer to the threshold
def count_occurrences(instance, array, nominal):
    occurrences = 0
    #if it's nominal
    if nominal == 1:
        for i in array:
            if i == instance:
                occurrences += 1
        return occurrences
    #if it's real
    if nominal == 0:
        for i in array:
            if i > instance:
                occurrences += 1
        return occurrences

In [39]:
count_occurrences(40, a.get_feature_data(1), 0)

45

In [40]:
def entropy(p):
    return -p*math.log2(p)

In [41]:
entropy(0.2)

0.46438561897747244

In [42]:
#inputs: class arff_data, feature_num(including the label), and threshold(only for real features)
def cumulative_entropy(arffdata, feature_num, threshold):
    feature = arffdata.all_attributes[feature_num]
    feature_data = arffdata.get_feature_data(feature_num)
    feature_value_list = feature.attribute_list
    
    if feature.type == "real":
        occurrences = count_occurrences(threshold, feature_data, 0)
        p = occurrences/len(feature_data)
        print(p)
        entropy = -p*math.log2(p)-(1-p)*math.log2(1-p)
        return entropy
    
    entropy = 0
    if feature.type == "nominal":
        for i in feature_value_list:
            p = count_occurrences(i, feature_data, 1)/len(feature_data)
            print(i, p)
            if p == 0:
                continue
            entropy += -p*math.log2(p)
        return entropy

In [43]:
cumulative_entropy(a, 8, 0)

+ 0.2874015748031496
- 0.7125984251968503


0.8653409924402706

In [96]:
#given the arff data and feature number, return a list of candidate splits
def determine_candidate_split(arffdata, feature_num):
    feature_value_list = arffdata.get_feature_data(feature_num)
    
    if arffdata.all_attributes[feature_num].type == "real":
        feature_value_list = sorted(feature_value_list)
        candidates = []
        for i in range(len(feature_value_list)-1):        
            candidates.append((feature_value_list[i]+feature_value_list[i+1])/2)
        return (list(set(candidates)))
    
    if arffdata.all_attributes[feature_num].type == "nominal":
        candidates = arffdata.all_attributes[feature_num].attribute_list
        return candidates

In [103]:
determine_candidate_split(a, 8)

['+', '-']